In [ ]:
%pip install torchaudio

In [1]:
import render
import rooms.dataset
import evaluate
import torch
import sounddevice as sd
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import metrics

In [2]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device  = "cpu"

In [3]:
D_0 = rooms.dataset.dataLoader("classroomBase")
#D

In [4]:
D_1 = rooms.dataset.dataLoader("classroomAddedPanel")

## Original audio

In [ ]:
sample_rate = 48000

sd.play(D_0.music_dls[0][0], sample_rate)

sd.wait()


## Base

In [5]:
print("Source position: ", D_0.speaker_xyz)
print("Listener position: ", D_0.xyzs[0])

Source position:  [3.5838 5.723  1.2294]
Listener position:  [2.4241125 0.581025  0.32     ]


In [5]:
L_0 = render.get_listener(source_xyz=D_0.speaker_xyz, listener_xyz = D_0.xyzs[0], surfaces=D_0.all_surfaces, 
                                                speed_of_sound=D_0.speed_of_sound,
                                                parallel_surface_pairs=D_0.parallel_surface_pairs,
                                                max_order=D_0.max_order, max_axial_order=D_0.max_axial_order )

Considered Paths:	59049
Total Considered Paths, after Axial:	59079
Valid Paths:	850


In [6]:
import torch
print(torch.version.cuda)  # Mostra la versione di CUDA compatibile
print(torch.cuda.is_available())  # Controlla se CUDA è disponibile


12.1
True


In [7]:
R_0 = render.Renderer(n_surfaces=len(D_0.all_surfaces))

In [8]:
pt_file = torch.load('./models/classroomBase/weights.pt', map_location=device)
R_0.energy_vector = nn.Parameter(pt_file['model_state_dict']['energy_vector'])
R_0.source_response = nn.Parameter(pt_file['model_state_dict']['source_response'])
R_0.directivity_sphere = nn.Parameter(pt_file['model_state_dict']['directivity_sphere'])
R_0.decay = nn.Parameter(pt_file['model_state_dict']['decay'])
R_0.RIR_residual = nn.Parameter(pt_file['model_state_dict']['RIR_residual'])
R_0.spline_values = nn.Parameter(pt_file['model_state_dict']['spline_values'])

In [9]:
RIR_0 = R_0.render_RIR(L_0)

In [10]:
RIR_0

tensor([5.3448e-10, 3.6882e-06, 5.5119e-06,  ..., 6.9838e-06, 6.3781e-06,
        6.7109e-06], grad_fn=<AddBackward0>)

In [11]:
torch.cuda.empty_cache()

In [29]:
RIR_by_direction_0 = R_0.render_RIR_by_directions(L_0)

In [30]:
len(RIR_by_direction_0[0]['responses'])

312

In [31]:
#trying with less data
RIR_by_direction_0[0]['responses'] = RIR_by_direction_0[0]['responses'][:100]

In [ ]:
len(RIR_by_direction_0[0]['responses'])

In [33]:
loss = metrics.training_loss_considering_directionality(RIR_by_direction_0, RIR_by_direction_0)
loss

In [ ]:
result = sum(value for value in RIR_by_direction_0.values())
result

In [ ]:
zeros = RIR_0
for RIR in RIR_by_direction_0.values():
    zeros = zeros - RIR

zeros

In [ ]:
print(RIR_0.unsqueeze(0).shape)
print(D_0.music_dls[0:1,...].shape)

In [ ]:
music_0 = evaluate.render_music(RIR_0.unsqueeze(0), D_0.music_dls[0:1,...], device='cpu')[0][0]

In [ ]:
print(music_0.shape)

In [ ]:
plt.plot(music_0)
plt.title('Music')
plt.show()

In [ ]:
sample_rate = 48000

sd.play(music_0, sample_rate)

sd.wait()


## added panel

In [ ]:
print("Source position: ", D_1.speaker_xyz)
print("Listener position: ", D_1.xyzs[0])

In [ ]:
L_1 = render.get_listener(source_xyz=D_1.speaker_xyz, listener_xyz = D_1.xyzs[0], surfaces=D_1.all_surfaces, 
                                                speed_of_sound=D_1.speed_of_sound,
                                                parallel_surface_pairs=D_1.parallel_surface_pairs,
                                                max_order=D_1.max_order, max_axial_order=D_1.max_axial_order )

In [ ]:
R_1 = render.Renderer(n_surfaces=len(D_1.all_surfaces))

In [ ]:
modified_energy_vector = torch.tensor([[[-0.5305, -0.4278, -0.2579, -0.3598, -1.6096, -0.7284, -1.8826,
          -1.6226, -1.5753, -1.9941],
         [ 0.5305,  0.4278,  0.2579,  0.3598,  1.6096,  0.7284,  1.8826,
           1.6226,  1.5753,  1.9941]],

        [[-2.4384,  0.8858, -0.2965, -1.5742, -0.2930, -1.9829, -1.7962,
          -1.9131, -0.0154,  0.2472],
         [ 2.4384, -0.8858,  0.2965,  1.5742,  0.2930,  1.9829,  1.7962,
           1.9131,  0.0154, -0.2472]],

        [[-0.9421, -0.6755, -3.6006, -0.7103, -0.8803, -1.3149, -2.2866,
          -2.6399, -2.8685, -2.1082],
         [ 0.9421,  0.6755,  3.6006,  0.7103,  0.8803,  1.3149,  2.2866,
           2.6399,  2.8685,  2.1082]],

        [[-4.0914, -2.7232, -0.8506, -2.3887, -3.6858, -1.3391, -2.6380,
          -1.8897, -2.1227, -3.5193],
         [ 4.0914,  2.7232,  0.8506,  2.3887,  3.6858,  1.3391,  2.6380,
           1.8897,  2.1227,  3.5193]],

        [[-1.1539, -0.4459, -0.3624, -0.7596, -0.7480, -1.5066, -0.9996,
          -2.1747, -0.6960, -2.0290],
         [ 1.1539,  0.4459,  0.3624,  0.7596,  0.7480,  1.5066,  0.9996,
           2.1747,  0.6960,  2.0290]],

        [[-1.7243, -1.1099, -0.0423, -1.1507, -1.2209, -0.4600, -2.0697,
          -2.2589, -2.4818, -1.5450],
         [ 1.7243,  1.1099,  0.0423,  1.1507,  1.2209,  0.4600,  2.0697,
           2.2589,  2.4818,  1.5450]],

        [[-7.8657, -7.9064, -4.2665, -2.9527, -0.6743, -0.7566,  1.5479,
          -1.3406, -2.4598, -2.1531],
         [ 7.8657,  7.9064,  4.2665,  2.9527,  0.6743,  0.7566, -1.5479,
           1.3406,  2.4598,  2.1531]],

        [[-4.5917, -5.2178, -4.6895, -3.5287, -3.0246,  0.2444, -1.6953,
          -1.4810, -1.9807, -1.9190],
         [ 4.5917,  5.2178,  4.6895,  3.5287,  3.0246, -0.2444,  1.6953,
           1.4810,  1.9807,  1.9190]],

        [[-6.7398, -2.8415, -3.7433, -0.8007, -2.0138, -1.9061, -2.0395,
          -2.8121, -3.1212, -2.7330],
         [ 6.7398,  2.8415,  3.7433,  0.8007,  2.0138,  1.9061,  2.0395,
          2.8121,  3.1212,  2.7330]],
           
        [[-6.7398, -2.8415, -3.7433, -0.8007, -2.0138, -1.9061, -2.0395,
          -2.8121, -3.1212, -2.7330],
         [ 6.7398,  2.8415,  3.7433,  0.8007,  2.0138,  1.9061,  2.0395,
          2.8121,  3.1212,  2.7330]]])

In [ ]:
pt_file = torch.load('./models/classroomBase/weights.pt', map_location=device)
R_1.energy_vector = nn.Parameter(modified_energy_vector)
R_1.source_response = nn.Parameter(pt_file['model_state_dict']['source_response'])
R_1.directivity_sphere = nn.Parameter(pt_file['model_state_dict']['directivity_sphere'])
R_1.decay = nn.Parameter(pt_file['model_state_dict']['decay'])
R_1.RIR_residual = nn.Parameter(pt_file['model_state_dict']['RIR_residual'])
R_1.spline_values = nn.Parameter(pt_file['model_state_dict']['spline_values'])

In [ ]:
RIR_1 = R_1.render_RIR(L_1)

In [ ]:
print(RIR_1.unsqueeze(0).shape)
print(D_1.music_dls[0:1,...].shape)

In [ ]:
music_1 = evaluate.render_music(RIR_1.unsqueeze(0), D_1.music_dls[0:1,...], device='cpu')[0][0]

In [ ]:
plt.plot(music_1)
plt.title('Music')
plt.show()

In [ ]:
sample_rate = 48000

sd.play(music_1, sample_rate)

sd.wait()